# 日期数据

## 格式转换


In [4]:
import pandas as pd

df_kindsof_date = pd.DataFrame({
    '日期': ['07-Oct-24', '10/07/2024', '2024.10.07', '2024/10/07', '20241007']
})

df_kindsof_date['新日期'] = pd.to_datetime(df_kindsof_date['日期'])
df_kindsof_date

C:\Users\lianb\AppData\Local\Temp\ipykernel_20596\3492425423.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_kindsof_date['新日期'] = pd.to_datetime(df_kindsof_date['日期'])


,日期,新日期
0,07-Oct-24,2024-10-07
1,10/07/2024,2024-10-07
2,2024.10.07,2024-10-07
3,2024/10/07,2024-10-07
4,20241007,2024-10-07


In [5]:
print(type(df_kindsof_date.loc[1, '日期']))
print(type(df_kindsof_date.loc[1, '新日期']))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


# 时间数据

## 重采样

In [8]:
import pandas as pd

s = pd.Series(
    range(9),
    index = pd.date_range('2024-10-07', periods=9, freq='min')
)
s

2024-10-07 00:00:00    0
2024-10-07 00:01:00    1
2024-10-07 00:02:00    2
2024-10-07 00:03:00    3
2024-10-07 00:04:00    4
2024-10-07 00:05:00    5
2024-10-07 00:06:00    6
2024-10-07 00:07:00    7
2024-10-07 00:08:00    8
Freq: min, dtype: int64

In [12]:
s.resample('3min').sum()

2024-10-07 00:00:00     3
2024-10-07 00:03:00    12
2024-10-07 00:06:00    21
Freq: 3min, dtype: int64